# Visualización de datos geoespaciales con Cartopy 🌍

Este notebook introduce el uso de **Cartopy** para visualizar datos espaciales. 
Si encuentran ejemplos antiguos en Basemap, la lógica es bastante parecida.

Veremos:
- Cómo visualizar datos geoespaciales a través de mapas con distintas proyecciones.
- Elementos y herramientas del gráfico (línea de costa, barras de colores y rangos, cuadrícula en los ejes).
- Diferentes formas de visualizar nuestros datos (`pcolormesh`, `contourf` y `contour`).

El notebook está pensado como una herramienta práctica para aprender y luego consultar cuando lo necesiten.

## Diferencia entre **Matplotlib**, **Basemap** y **Cartopy**

- **Matplotlib**: grafica en coordenadas cartesianas (x, y).  
- **Basemap**: fue la primera extensión de matplotlib para mapas, hoy está en desuso.  
- **Cartopy**: la librería actual recomendada; integra directamente con matplotlib y soporta múltiples proyecciones.  

Cartopy es más flexible y tiene soporte activo.

## 1. Instalación

Para instalar las librerías necesarias, escribimos en la terminal dentro de nuestro entorno de trabajo:

```bash
pip install xarray matplotlib cartopy cmocean
```

## 2. Importación de librerías necesarias

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean

## 3. Abrir nuestro archivo de datos con formato NetCDF

Usaremos un archivo NetCDF que contiene información de **temperatura superficial del mar (SST)** de MODIS.

Ejemplo: `sst_modis.nc`, con coordenadas `lat`, `lon` y `time`. Podemos seleccionar un instante de tiempo para graficar el campo espacial:

In [ ]:
# Abrir dataset NetCDF
ds = xr.open_dataset("sst_modis.nc")

ds.info()
ds.coords

In [ ]:
# Seleccionar variable y un instante de tiempo
sst = ds['sst'].isel(time=0)

lon = ds.lon.values
lat = ds.lat.values

## 4. Visualización básica con Matplotlib (sin proyección)

Usaremos `pcolormesh` para una visualización rápida de los datos.

In [ ]:
plt.pcolormesh(lon, lat, sst)

In [ ]:
plt.figure(figsize=(8,5))
plt.pcolormesh(lon, lat, sst, cmap=cmocean.cm.thermal)
plt.colorbar(label="SST (°C)")
plt.title("Visualización básica con matplotlib (sin proyección)")
plt.xlabel("Longitud")
plt.ylabel("Latitud")
plt.show()

## 5. Visualización básica con Cartopy

Graficaremos el campo de temperatura en una proyección **PlateCarree** (la más simple).

💡 Nota pedagógica: `transform=ccrs.PlateCarree()` indica que las coordenadas de los datos están en lat/lon. Esto es importante para que Cartopy proyecte correctamente los datos sobre la proyección del mapa.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.PlateCarree())
sst.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                    cbar_kwargs={'label': 'SST (°C)'})
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.set_title("Temperatura superficial del mar (MODIS) - Proyección PlateCarree")
plt.show()

## Ejemplo con otra proyección: Miller

La **proyección Miller** es similar a Mercator, pero corrige ligeramente la distorsión en latitudes altas. Útil para mapas globales.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.Miller())
sst.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                    cbar_kwargs={'label': 'SST (°C)'})
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.set_title("Temperatura superficial del mar (MODIS) - Proyección Miller")
plt.show()

## 6. Diferencias entre `pcolormesh`, `contour` y `contourf`

- **pcolormesh**: colorea cada celda con el valor correspondiente (ideal para datos de grilla).
- **contour**: dibuja sólo las líneas de contorno (isolíneas), sin color de relleno.
- **contourf**: crea polígonos coloreados interpolando entre valores (útil para visualizar gradientes suaves).

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,5), subplot_kw={'projection': ccrs.PlateCarree()})

# pcolormesh
sst.plot.pcolormesh(ax=axes[0], transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                    cbar_kwargs={'label': 'SST (°C)'})
axes[0].coastlines()
axes[0].set_title("pcolormesh")

# contourf
sst.plot.contourf(ax=axes[1], transform=ccrs.PlateCarree(), cmap=cmocean.cm.thermal,
                  cbar_kwargs={'label': 'SST (°C)'})
axes[1].coastlines()
axes[1].set_title("contourf")

# contour
sst.plot.contour(ax=axes[2], transform=ccrs.PlateCarree(), colors='black')
axes[2].coastlines()
axes[2].set_title("contour")

plt.show()

## Figura Final Mejorada

Ejemplo de mapa con estética más cuidada y uso de herramientas de Cartopy.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111, projection=ccrs.Miller())

# Título
ax.set_title("Temperatura superficial del mar\nMODIS Terra (2000-2024)")

# Graficar datos
pcm = ax.pcolormesh(lon, lat, sst, transform=ccrs.PlateCarree(), vmin=4., vmax=14., cmap=cmocean.cm.thermal)

# Línea de costa y bordes
coast = cfeature.GSHHSFeature(scale="l")
ax.add_feature(coast, linewidth=0.25, facecolor='.65')

# Gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='.25', linestyle='-.', zorder=2)
gl.top_labels = False 
gl.right_labels = False

# Barra de colores
cb = plt.colorbar(pcm, extend="both")
cb.set_label("SST (°C)", rotation=0, ha="left")

plt.show()

## Ejercicios

- Cambiar la zona de interés usando `ax.set_extent()` y adaptar el rango de valores de temperatura.
- Cambiar la proyección de la figura.
- Añadir fronteras y ríos usando `cartopy.feature`.